In [1]:
#### Constants in the scheme
C = var("C", latex_name="\\mathcal{C}") ## Courant number
s1 = var("s1", latex_name="s_1")        ## First relaxation parameter (useless)
s2 = var("s2", latex_name="s_2")        ## Second relaxation parameter
s3 = var("s3", latex_name="s_3")        ## Third relaxation parameter

assume(C,'real')
assume(s2,'real')
assume(s3,'real')

#assume(s2>0)
#assume(s3>0)

#### Pieces involved in the scheme
q = 3 ## Number of discrete velocities
c1 = 0  ## First discrete velocity
c2 = 1  ## Second discrete velocity
c3 = -1 ## Third discrete velocity
r = 1 ## Stencil to the left
p = 1 ## Stencil to the right
M = matrix([[1,  1,  1], \
            [0,  1, -1], \
            [0,  1,  1]]) ## Moment matrix
eps1 = 1    ## First equilibrium coefficient (useless)
eps2 = C    ## Second equilibrium coefficient
eps3 = C**2 ## Third equilibrium coefficient

#### Construction of the objects involved in the analysis
k = var("kappa", latex_name="\\kappa") 

Minv = M.inverse()
K = identity_matrix(q)+diagonal_matrix([s1, s2, s3])\
        *(matrix([[eps1], [eps2], [eps3]])*matrix([[1, 0, 0]])-identity_matrix(q))
Ehat = M*diagonal_matrix([k**(-c1), k**(-c2), k**(-c3)])*Minv*K #### Bulk matrix scheme "Fourier"
for i in range(q):
    for j in range(q):
        Ehat[i, j] = Ehat[i, j].full_simplify().collect(k)
    
z = var("z", latex_name="z") 

charEq = (z*identity_matrix(q)-Ehat).determinant().full_simplify().collect(k) #### Characteristic equation
dm1 = charEq.coefficient(k, -1).collect(z) ### Coefficient d_{-1} of the characteristic equation
d0  = charEq.coefficient(k,  0).collect(z) ### Coefficient d_{0} of the characteristic equation
dp1 = charEq.coefficient(k,  1).collect(z) ### Coefficient d_{1} of the characteristic equation

Em1 = matrix(SR, q, q) ### Matrix for the point -1
E0  = matrix(SR, q, q) ### Matrix for the point 0
Ep1 = matrix(SR, q, q) ### Matrix for the point 1
for i in range(q):
    for j in range(q):
        Em1[i, j] = Ehat[i, j].coefficient(k, -1)
        E0[i, j]  = Ehat[i, j].coefficient(k,  0)
        Ep1[i, j] = Ehat[i, j].coefficient(k,  1)
Lz = - Em1 + (z*identity_matrix(q)-E0)*k - Ep1*k**2 ### Matrix polynomial in k
print('!! Sanity check: the result must be zero.')
pretty_print((Lz.determinant()/k**(r*q)-charEq).full_simplify()) ### Sanity check

kplus  = ((-d0+sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the plus
kminus = ((-d0-sqrt(d0**2-4*dp1*dm1))/2/dp1).full_simplify() ### Root of the char eq with the minus


Pi = (dm1/dp1).full_simplify() ### This is the product of the roots in k 

!! Sanity check: the result must be zero.


0

In [2]:
##### Boundary conditions that we analyze for any value of C and s_2 and s_3
bdMat_BB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, 1], [0, 0, k]])*Minv*K ## Bounce back
bdMat_ABB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, -1], [0, 0, k]])*Minv*K ## Anti Bounce back
bdMat_TwoABB = z*identity_matrix(q) - M*matrix([[1, 0, 0], [-1, 0, -k], [0, 0, k]])*Minv*K ## Two steps Anti Bounce back
bdMat_sigma1 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 1, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 1
bdMat_sigma2 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 2-k, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 2
bdMat_sigma3 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 3-3*k+k**2, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 3
bdMat_sigma4 = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 4-6*k+4*k**2-k**3, 0], [0, 0, k]])*Minv*K ## Extrapolation sigma = 4
bdMat_kinDir = z*identity_matrix(q) - M*matrix([[1, 0, 0], [0, 0, 0], [0, 0, k]])*Minv*K ## Kinetic dirichlet

In [3]:
##### We find the critical (z, k) eigenvalues between boundary and bulk
print("Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_BB.determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
#pretty_print(solve([charEq.full_simplify(), bdMat_ABB.determinant().full_simplify()], (z, k))) ### Gets stuck
kABB = solve(bdMat_ABB.determinant().full_simplify(), k)[0].rhs().full_simplify()
zABB = solve(charEq.full_simplify().subs(k==kABB), z)
pretty_print(zABB)
phi2 = ((z-zABB[0].rhs())*(z-zABB[1].rhs())).expand().full_simplify().collect(z)
pretty_print("phi2 = "+str(phi2))
phi2Star = (z**2*phi2.subs(z==1/z)).full_simplify().collect(z)
pretty_print(phi2Star)
pretty_print("phi2Star = "+str(phi2Star))
pretty_print("phi2(0) = "+str(phi2.subs(z==0)))
pretty_print("phi2Star(0) = "+str(phi2Star.subs(z==0)))
phi1 = ((phi2Star.subs(z==0)*phi2 - phi2.subs(z==0)*phi2Star)/z).full_simplify().collect(z)
print("Root of phi1")
pretty_print(solve(phi1, z))
print("Two steps anti Bounce back")
pretty_print(solve([charEq.full_simplify(), bdMat_TwoABB.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma1.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma2.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma3.determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([charEq.full_simplify(), bdMat_sigma4.determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([charEq.full_simplify(), bdMat_kinDir.determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == s2 - 1, kappa == -1], [z == -s3 + 1, kappa == 1], [z == 1, kappa == ((C - 1)*s2 - 2*C)/((C + 1)*s2 - 2*C)], [z == -((C*s2 - C - 1)*s3 + s2)/((C + 1)*s3 - s2), kappa == -((C*s2 - C - 1)*s3 + s2)/(s2^2 - ((C + 1)*s2 - C - 1)*s3 - s2)]]

Anti Bounce back


[z == -1/2*(2*C^2 - 1)*s3 - 1/2*sqrt((4*C^4 - 4*C^2 + 1)*s3^2 - 4*s3 + 4), z == -1/2*(2*C^2 - 1)*s3 + 1/2*sqrt((4*C^4 - 4*C^2 + 1)*s3^2 - 4*s3 + 4), z == -((s2*s3 - s3)*C + s2 - s3)/(C*s3 - s2 + s3), z == -s2 + 1]

'phi2 = (2*C^2 - 1)*s3*z + z^2 + s3 - 1'

...

[[z == 1, kappa == 1], [z == -s3 + 1, kappa == 1], [z == -s2 + 1, kappa == 1], [z == -((C*s2 - C + 1)*s3 - s2)/((C - 1)*s3 + s2), kappa == -((C^2*s2 - C^2 + C)*s3^2 - s2^2 + (C*s2^2 - (2*C - 1)*s2)*s3)/((C^2 - (C^2 - 1)*s2 - C)*s3^2 + s2^2 - ((C + 1)*s2^2 - (2*C - 1)*s2)*s3)], [z == -((C*s2 - C - 1)*s3 + s2)/((C + 1)*s3 - s2), kappa == -((C*s2 - C - 1)*s3 + s2)/(s2^2 - ((C + 1)*s2 - C - 1)*s3 - s2)], [z == 0, kappa == 0]]

Extrapolation sigma = 4


[[z == 1, kappa == 1], [z == -s3 + 1, kappa == 1], [z == -s2 + 1, kappa == 1], [z == -((C*s2 - C + 1)*s3 - s2)/((C - 1)*s3 + s2), kappa == -((C^2*s2 - C^2 + C)*s3^2 - s2^2 + (C*s2^2 - (2*C - 1)*s2)*s3)/((C^2 - (C^2 - 1)*s2 - C)*s3^2 + s2^2 - ((C + 1)*s2^2 - (2*C - 1)*s2)*s3)], [z == -((C*s2 - C - 1)*s3 + s2)/((C + 1)*s3 - s2), kappa == -((C*s2 - C - 1)*s3 + s2)/(s2^2 - ((C + 1)*s2 - C - 1)*s3 - s2)], [z == 0, kappa == 0]]

Kinetic Dirichlet


[[z == 0, kappa == 0], [z == -((C*s2 - C + 1)*s3 - s2)/((C - 1)*s3 + s2), kappa == -((C^2*s2 - C^2 + C)*s3^2 - s2^2 + (C*s2^2 - (2*C - 1)*s2)*s3)/((C^2 - (C^2 - 1)*s2 - C)*s3^2 + s2^2 - ((C + 1)*s2^2 - (2*C - 1)*s2)*s3)], [z == -((C*s2 - C - 1)*s3 + s2)/((C + 1)*s3 - s2), kappa == -((C*s2 - C - 1)*s3 + s2)/(s2^2 - ((C + 1)*s2 - C - 1)*s3 - s2)]]

In [4]:
##### Sanity check : we set some values for s_2 and C and see if we find more
##### Sometimes sagemath does not give all the roots
s2_val = 11/10
s3_val = 105/100
C_val = -1/3

print("Bounce back")
pretty_print(solve([charEq.subs(s2=s2_val, s3=s3_val, C=C_val).full_simplify(), bdMat_BB.subs(s2=s2_val, s3=s3_val, C=C_val).determinant().full_simplify()], (z, k)))
print("Anti Bounce back")
#pretty_print(solve([charEq.full_simplify(), bdMat_ABB.determinant().full_simplify()], (z, k))) ### Gets stuck
kABB = solve(bdMat_ABB.subs(s2=s2_val, s3=s3_val, C=C_val).determinant().full_simplify(), k)[0].rhs().full_simplify()
zABB = solve(charEq.subs(s2=s2_val, s3=s3_val, C=C_val).full_simplify().subs(k==kABB), z)
pretty_print(zABB)
phi2 = ((z-zABB[0].rhs())*(z-zABB[1].rhs())).expand().full_simplify().collect(z)
pretty_print("phi2 = "+str(phi2))
phi2Star = (z**2*phi2.subs(z==1/z)).full_simplify().collect(z)
pretty_print(phi2Star)
pretty_print("phi2Star = "+str(phi2Star))
pretty_print("phi2(0) = "+str(phi2.subs(z==0)))
pretty_print("phi2Star(0) = "+str(phi2Star.subs(z==0)))
phi1 = ((phi2Star.subs(z==0)*phi2 - phi2.subs(z==0)*phi2Star)/z).full_simplify().collect(z)
print("Root of phi1")
pretty_print(solve(phi1, z))
print("Two steps anti Bounce back")
pretty_print(solve([charEq.subs(s2=s2_val, s3=s3_val, C=C_val).full_simplify(), bdMat_TwoABB.subs(s2=s2_val, s3=s3_val, C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 1")
pretty_print(solve([charEq.subs(s2=s2_val, s3=s3_val, C=C_val).full_simplify(), bdMat_sigma1.subs(s2=s2_val, s3=s3_val, C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 2")
pretty_print(solve([charEq.subs(s2=s2_val, s3=s3_val, C=C_val).full_simplify(), bdMat_sigma2.subs(s2=s2_val, s3=s3_val, C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 3")
pretty_print(solve([charEq.subs(s2=s2_val, s3=s3_val, C=C_val).full_simplify(), bdMat_sigma3.subs(s2=s2_val, s3=s3_val, C=C_val).determinant().full_simplify()], (z, k)))
print("Extrapolation sigma = 4")
pretty_print(solve([charEq.subs(s2=s2_val, s3=s3_val, C=C_val).full_simplify(), bdMat_sigma4.subs(s2=s2_val, s3=s3_val, C=C_val).determinant().full_simplify()], (z, k)))
print("Kinetic Dirichlet")
pretty_print(solve([charEq.subs(s2=s2_val, s3=s3_val, C=C_val).full_simplify(), bdMat_kinDir.subs(s2=s2_val, s3=s3_val, C=C_val).determinant().full_simplify()], (z, k)))

Bounce back


[[z == 0, kappa == 0], [z == 1, kappa == (-4/7)], [z == (-1/20), kappa == 1], [z == (1/10), kappa == -1], [z == (3/80), kappa == (-3/8)]]

Anti Bounce back


[z == (3/80), z == (-1/10), z == (3/4), z == (1/15)]

'phi2 = z^2 + 1/16*z - 3/800'

...

[[z == 1, kappa == 1], [z == (-1/20), kappa == 1], [z == (-1/10), kappa == 1], [z == (-17/60), kappa == (85/8)], [z == (3/80), kappa == (-3/8)], [z == 0, kappa == 0]]

Extrapolation sigma = 4


[[z == 1, kappa == 1], [z == (-1/20), kappa == 1], [z == (-1/10), kappa == 1], [z == (-17/60), kappa == (85/8)], [z == (3/80), kappa == (-3/8)], [z == 0, kappa == 0]]

Kinetic Dirichlet


[[z == 0, kappa == 0], [z == (-17/60), kappa == (85/8)], [z == (3/80), kappa == (-3/8)]]

In [5]:
##### Construction of the eigenvector for k = 0
pretty_print(Lz.subs(k==0).transpose().kernel())

phi01 = matrix([[((s3-1)*Lz.subs(k==0).transpose().kernel().basis()[0][0]).full_simplify()], \
                [((s3-1)*Lz.subs(k==0).transpose().kernel().basis()[0][1]).full_simplify()], \
                [((s3-1)*Lz.subs(k==0).transpose().kernel().basis()[0][2]).full_simplify()]])
phi02 = matrix([[((s3-1)*Lz.subs(k==0).transpose().kernel().basis()[1][0]).full_simplify()], \
                [((s3-1)*Lz.subs(k==0).transpose().kernel().basis()[1][1]).full_simplify()], \
                [((s3-1)*Lz.subs(k==0).transpose().kernel().basis()[1][2]).full_simplify()]])

Vector space of degree 3 and dimension 2 over Symbolic Ring
Basis matrix:
[                       1                        0 (C^2*s3 + C*s2)/(s3 - 1)]
[                       0                        1       -(s2 - 1)/(s3 - 1)]

In [6]:
#### We make natural assumptions
assume(s2>0)
assume(s2<=2)
assume(s3>0)
assume(s3<=2)

In [7]:
#### Critical point z = 1, k = Pi for C<0

assume(C<0)
print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).expand().full_simplify()) ### This order is the only one which does not block the computation with these assumptions
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).expand().full_simplify())

ks = kplus.full_simplify()

### I do not compute Taylor expansions in z, too expensive (with three free parameters)
### Still, the eigenvector does not have poles.

LzLim = Lz.subs(k==Pi).subs(z==1)

for i in range(q):
    for j in range(q):
        LzLim[i, j] = LzLim[i, j].full_simplify()
     
phiSLim = matrix([[LzLim.transpose().kernel().basis()[0][0]], \
                  [LzLim.transpose().kernel().basis()[0][1].full_simplify()], \
                  [LzLim.transpose().kernel().basis()[0][2].full_simplify()]])

bd00  = matrix(SR, q, q)
bd01 = matrix(SR, q, q)

Bs = matrix(SR, q, q) #### Boundary matrix on the stable subspace

#### Bounce back 
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_BB[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*Pi).subs(z==1)*phiSLim ### First column
Bs[:, 1] = (bd00).subs(z==1)*phi01 ### Second column
Bs[:, 2] = (bd00).subs(z==1)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
print("Boundary matrix on the stable subspace for bounce back at z = 1")
pretty_print(Bs)

Value of kminus at z=1


1

Value of kplus at z=1


((C - 1)*s2 - 2*C)/((C + 1)*s2 - 2*C)

Boundary matrix on the stable subspace for bounce back at z = 1


[                    0                     0                     0]
[                    0        C*s2*s3 - C*s2 -(s2 - 2)*s3 + s2 - 2]
[                    0       (C^2 + C*s2)*s3          -(s2 - 1)*s3]

In [13]:
#### Critical point z = 1, k = 1 for C>0
forget()
assume(C>0)
#### We make natural assumptions
assume(s2>0)
assume(s2<=2)
assume(s3>0)
assume(s3<=2)

print("Value of kminus at z=1")
pretty_print(kminus.subs(z==1).expand().full_simplify()) ### This order is the only one which does not block the computation with these assumptions
print("Value of kplus at z=1")
pretty_print(kplus.subs(z==1).expand().full_simplify())

ks = kplus.full_simplify()


### I do not compute Taylor expansions in z, too expensive (with three free parameters)
### Still, the eigenvector does not have poles.

LzLim = Lz.subs(k==1).subs(z==1)

for i in range(q):
    for j in range(q):
        LzLim[i, j] = LzLim[i, j].full_simplify()
     
phiSLim = matrix([[LzLim.transpose().kernel().basis()[0][0]], \
                  [LzLim.transpose().kernel().basis()[0][1].full_simplify()], \
                  [LzLim.transpose().kernel().basis()[0][2].full_simplify()]])

bd00  = matrix(SR, q, q)
bd01 = matrix(SR, q, q)
bd02 = matrix(SR, q, q)
bd03 = matrix(SR, q, q)

Bs = matrix(SR, q, q) #### Boundary matrix on the stable subspace

#### Extrapolation sigma = 1
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma1[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*1).subs(z==1)*phiSLim ### First column
Bs[:, 1] = (bd00).subs(z==1)*phi01 ### Second column
Bs[:, 2] = (bd00).subs(z==1)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
print("Boundary matrix on the stable subspace for extrapolation sigma = 1 at z = 1")
pretty_print(Bs)

#### Extrapolation sigma = 2
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma2[i, j].full_simplify().collect(k).coefficient(k, 1)
        
Bs[:, 0] = (bd00 + bd01*1).subs(z==1)*phiSLim ### First column
Bs[:, 1] = (bd00).subs(z==1)*phi01 ### Second column
Bs[:, 2] = (bd00).subs(z==1)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
print("Boundary matrix on the stable subspace for extrapolation sigma = 2 at z = 1")
pretty_print(Bs)

#### Extrapolation sigma = 3
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma3[i, j].full_simplify().collect(k).coefficient(k, 2)
        
Bs[:, 0] = (bd00 + bd01*1 + bd02*1**2).subs(z==1)*phiSLim ### First column
Bs[:, 1] = (bd00).subs(z==1)*phi01 ### Second column
Bs[:, 2] = (bd00).subs(z==1)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
print("Boundary matrix on the stable subspace for extrapolation sigma = 3 at z = 1")
pretty_print(Bs)

#### Extrapolation sigma = 4
for i in range(q):
    for j in range(q):
        bd00[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 0)
        bd01[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 1)
        bd02[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 2)
        bd03[i, j] = bdMat_sigma4[i, j].full_simplify().collect(k).coefficient(k, 3)
        
Bs[:, 0] = (bd00 + bd01*1 + bd02*1**2 + bd03*1**3).subs(z==1)*phiSLim ### First column
Bs[:, 1] = (bd00).subs(z==1)*phi01 ### Second column
Bs[:, 2] = (bd00).subs(z==1)*phi02 ### Third column

for i in range(q):
    for j in range(q):
        Bs[i, j] = Bs[i, j].full_simplify()
print("Boundary matrix on the stable subspace for extrapolation sigma = 4 at z = 1")
pretty_print(Bs)

Value of kminus at z=1


((C - 1)*s2 - 2*C)/((C + 1)*s2 - 2*C)

Value of kplus at z=1


1

Boundary matrix on the stable subspace for extrapolation sigma = 1 at z = 1


...

[                   0      -C*s2*s3 + C*s2 (s2 - 1)*s3 - s2 + 1]
[                   0                    0               s3 - 1]
[                   0        C^2*s3 + C*s2              -s2 + 1]

Boundary matrix on the stable subspace for extrapolation sigma = 3 at z = 1


[                   0      -C*s2*s3 + C*s2 (s2 - 1)*s3 - s2 + 1]
[                   0                    0               s3 - 1]
[                   0        C^2*s3 + C*s2              -s2 + 1]

Boundary matrix on the stable subspace for extrapolation sigma = 4 at z = 1


[                   0      -C*s2*s3 + C*s2 (s2 - 1)*s3 - s2 + 1]
[                   0                    0               s3 - 1]
[                   0        C^2*s3 + C*s2              -s2 + 1]